In [1]:
#Importamos paquetes, la idea es procesaro los distintos datos diarios que encontramos entre 01-01-2023 y 31-05-2023, para distintos dias de extraccion y temas, distintos paises y comunidades autonomas. 

import os
import pandas as pd
import re

In [2]:

# Ruta base donde están las carpetas de extracciones
base_path = r"C:\Users\34645\Desktop\projects\Obj1_GT\data\datos_navegador"

# Lista de códigos ISO de los países de interés
iso_codes = ["ES", "IT", "UK", "FR", "PT", "DE"]

# Diccionario para almacenar la numeración de extracciones por país
extraction_numbers_by_country = {}

# Listas para almacenar los DataFrames por set de temas
dataframes_set1 = []
dataframes_set2 = []

# Buscar todas las carpetas de extracciones dentro de la ruta base y ordenarlas cronológicamente
extraction_folders = sorted(
    [os.path.join(base_path, folder) for folder in os.listdir(base_path) if folder.startswith("extraccion_")]
)

for folder in extraction_folders:
    # Extraer la fecha de la carpeta
    extraction_date_match = re.search(r"extraccion_(\d{2})(\d{2})(\d{4})", folder)
    if extraction_date_match:
        extraction_date = f"{extraction_date_match.group(3)}-{extraction_date_match.group(2)}-{extraction_date_match.group(1)}"
    else:
        continue  # Si la carpeta no tiene fecha válida, saltarla

    # Obtener la lista de archivos CSV dentro de cada carpeta
    csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

    for file in csv_files:
        file_path = os.path.join(folder, file)

        # Determinar si es set1 o set2
        if "set1_temas" in file:
            tema_set = "set1"
        elif "set2_temas" in file:
            tema_set = "set2"
        else:
            continue  # Ignorar archivos que no sean de temas

        # Extraer el país desde el nombre del archivo (si no hay sufijo, es España)
        country_match = re.search(r"_(\w{2})\.csv$", file)
        if country_match:
            country_iso = country_match.group(1).upper()  # Extrae el ISO (ejemplo: 'FR', 'DE', 'UK')
        else:
            country_iso = "ES"  # Si no hay sufijo, asumimos España

        # **Asignar número de extracción correctamente**
        if country_iso not in extraction_numbers_by_country:
            # Si el país aparece por primera vez, su extracción es 1
            extraction_numbers_by_country[country_iso] = {extraction_date: 1}
        else:
            # Si el país ya había aparecido antes, contar correctamente las extracciones por fecha
            if extraction_date not in extraction_numbers_by_country[country_iso]:
                extraction_numbers_by_country[country_iso][extraction_date] = len(extraction_numbers_by_country[country_iso]) + 1

        # Obtener el número de extracción correcto para este país en esta fecha
        extraction_number = extraction_numbers_by_country[country_iso][extraction_date]

        # Intentar cargar el CSV ignorando la primera fila
        try:
            df = pd.read_csv(file_path, skiprows=1, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, skiprows=1, encoding="ISO-8859-1")

        # Renombrar la primera columna como "date" y convertirla a formato fecha
        df.rename(columns={df.columns[0]: "date"}, inplace=True)
        df["date"] = pd.to_datetime(df["date"], errors='coerce')

        # Limpiar los nombres de las columnas eliminando caracteres especiales
        df.columns = df.columns.str.replace(r'[^a-zA-Z0-9_áéíóúñÁÉÍÓÚÑ]', '_', regex=True)

        # Transformar a formato long (melt)
        df_long = df.melt(id_vars=["date"], var_name="tema", value_name="volumen_de_busqueda")

        # Agregar las columnas de metadatos
        df_long["extraccion"] = extraction_number  # Número de extracción basado en la primera aparición del país
        df_long["fecha_extraccion"] = extraction_date
        df_long["pais_region"] = country_iso  # Código ISO del país
        df_long["categoria"] = "temas"
        df_long["set"] = tema_set  # Indica si pertenece a set1 o set2

        # Guardar en la lista correspondiente
        if tema_set == "set1":
            dataframes_set1.append(df_long)
        else:
            dataframes_set2.append(df_long)

# Unir todos los DataFrames en sus respectivos sets
df_set1_temas = pd.concat(dataframes_set1, ignore_index=True) if dataframes_set1 else pd.DataFrame()
df_set2_temas = pd.concat(dataframes_set2, ignore_index=True) if dataframes_set2 else pd.DataFrame()

# Ordenar por país, fecha y extracción
df_set1_temas.sort_values(by=["pais_region", "date", "tema", "extraccion"], inplace=True)
df_set2_temas.sort_values(by=["pais_region", "date", "tema", "extraccion"], inplace=True)




In [3]:
df_set2_temas

,date,tema,volumen_de_busqueda,extraccion,fecha_extraccion,pais_region,categoria,set
7097,2023-01-01,Autobús___Aragón_,57,1,2025-03-18,AR,temas,set2
7399,2023-01-01,Automóvil___Aragón_,71,1,2025-03-18,AR,temas,set2
6795,2023-01-01,Google_Maps___Aragón_,11,1,2025-03-18,AR,temas,set2
7248,2023-01-01,Taxi___Aragón_,18,1,2025-03-18,AR,temas,set2
6946,2023-01-01,Tránsito_vehicular___Aragón_,0,1,2025-03-18,AR,temas,set2
...,...,...,...,...,...,...,...,...
14042,2023-05-31,Autobús___Reino_Unido_,38,1,2025-03-18,UK,temas,set2
14344,2023-05-31,Automóvil___Reino_Unido_,98,1,2025-03-18,UK,temas,set2
13740,2023-05-31,Google_Maps___Reino_Unido_,17,1,2025-03-18,UK,temas,set2
14193,2023-05-31,Taxi___Reino_Unido_,9,1,2025-03-18,UK,temas,set2


In [4]:

# Ruta base donde están las carpetas de extracciones
base_path = r"C:\Users\34645\Desktop\projects\Obj1_GT\data\datos_navegador"

# Diccionario con los códigos ISO de las comunidades autónomas
region_iso_codes = {
    "AR": "Aragón",
    "RI": "La Rioja",
    "EX": "Extremadura",
    "MD": "Comunidad de Madrid",
    "CT": "Cataluña"
}

# Diccionario para almacenar la numeración de extracciones por región/país
extraction_numbers_by_region = {}

# **CORRECCIÓN: Inicializamos correctamente dataframes_keywords como diccionario**
dataframes_keywords = {f"set{i}_long": [] for i in range(1, 7)}

# Buscar todas las carpetas de extracciones dentro de la ruta base y ordenarlas cronológicamente
extraction_folders = sorted(
    [os.path.join(base_path, folder) for folder in os.listdir(base_path) if folder.startswith("extraccion_")]
)

for folder in extraction_folders:
    # Extraer la fecha de la carpeta
    extraction_date_match = re.search(r"extraccion_(\d{2})(\d{2})(\d{4})", folder)
    if extraction_date_match:
        extraction_date = f"{extraction_date_match.group(3)}-{extraction_date_match.group(2)}-{extraction_date_match.group(1)}"
    else:
        continue  # Si la carpeta no tiene fecha válida, saltarla

    # Obtener la lista de archivos CSV dentro de cada carpeta
    csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

    for file in csv_files:
        file_path = os.path.join(folder, file)

        # Determinar qué set de keywords es y asegurarse de que coincida con las claves del diccionario
        set_match = re.search(r"(set\d+)_long", file)
        if set_match:
            keyword_set = f"{set_match.group(1)}_long"  # Asegurar que coincida con las claves del diccionario
        else:
            continue  # Ignorar archivos que no sean de keywords

        # Extraer la región desde el nombre del archivo (si no hay sufijo, es España total)
        region_match = re.search(r"_(\w{2})\.csv$", file)
        if region_match:
            region_iso = region_match.group(1).upper()  # Extrae el ISO (ejemplo: 'AR', 'MD', etc.)
            region_name = region_iso_codes.get(region_iso, region_iso)  # Convertir código a nombre
        else:
            region_iso = "ES"  # Si no hay sufijo, asumimos España
            region_name = "España"

        # **Asignar número de extracción correctamente**
        if region_iso not in extraction_numbers_by_region:
            # Si la región aparece por primera vez, su extracción es 1
            extraction_numbers_by_region[region_iso] = {extraction_date: 1}
        else:
            # Si la región ya había aparecido antes, contar correctamente las extracciones por fecha
            if extraction_date not in extraction_numbers_by_region[region_iso]:
                extraction_numbers_by_region[region_iso][extraction_date] = len(extraction_numbers_by_region[region_iso]) + 1

        # Obtener el número de extracción correcto para esta región en esta fecha
        extraction_number = extraction_numbers_by_region[region_iso][extraction_date]

        # Intentar cargar el CSV ignorando la primera fila
        try:
            df = pd.read_csv(file_path, skiprows=1, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, skiprows=1, encoding="ISO-8859-1")

        # Renombrar la primera columna como "date" y convertirla a formato fecha
        df.rename(columns={df.columns[0]: "date"}, inplace=True)
        df["date"] = pd.to_datetime(df["date"], errors='coerce')

        # Limpiar los nombres de las columnas eliminando caracteres especiales
        df.columns = df.columns.str.replace(r'[^a-zA-Z0-9_áéíóúñÁÉÍÓÚÑ]', '_', regex=True)

        # Transformar a formato long (melt)
        df_long = df.melt(id_vars=["date"], var_name="keyword", value_name="volumen_de_busqueda")

        # Agregar las columnas de metadatos
        df_long["extraccion"] = extraction_number  # Número de extracción basado en la primera aparición de la región
        df_long["fecha_extraccion"] = extraction_date
        df_long["region"] = region_iso  # Código ISO de la región
        df_long["region_nombre"] = region_name  # Nombre de la región
        df_long["categoria"] = "keywords"
        df_long["set"] = keyword_set  # Indica el set de keywords

        # Guardar en la lista correspondiente al set
        if keyword_set in dataframes_keywords:  # Verificar que la clave existe en el diccionario
            dataframes_keywords[keyword_set].append(df_long)
        else:
            print(f"Advertencia: {keyword_set} no encontrado en dataframes_keywords")

# Unir todos los DataFrames en sus respectivos sets
df_keywords_sets = {}
for key, data in dataframes_keywords.items():
    df_keywords_sets[key] = pd.concat(data, ignore_index=True) if data else pd.DataFrame()

# Ordenar por región, fecha y extracción
for key in df_keywords_sets:
    df_keywords_sets[key].sort_values(by=["region", "date", "keyword", "extraccion"], inplace=True)




In [5]:
df_keywords_sets

{'set1_long':             date                keyword  volumen_de_busqueda  extraccion  \
 3322  2023-01-01    bicicleta___Aragón_                   17           1   
 7097  2023-01-01    bicicleta___Aragón_                   17           2   
 3473  2023-01-01          bus___Aragón_                   47           1   
 7248  2023-01-01          bus___Aragón_                   50           2   
 3020  2023-01-01        coche___Aragón_                   78           1   
 ...          ...                    ...                  ...         ...   
 9965  2023-05-31      coche___La_Rioja_                   67           2   
 5586  2023-05-31       taxi___La_Rioja_                    0           1   
 10116 2023-05-31       taxi___La_Rioja_                    0           2   
 6039  2023-05-31  wikipedia___La_Rioja_                    0           1   
 10569 2023-05-31  wikipedia___La_Rioja_                    0           2   
 
       fecha_extraccion region region_nombre categoria       